# Zadanie 1

In [ ]:
# podział na train i test, przygotowanie danych
import numpy as np
X_train_parabolic = np.linspace(-50, 50, 26)
X_train_sin = np.linspace(0, 2, 21)
Y_train_parabolic = X_train_parabolic**2
Y_train_sin = np.sin((3*np.pi/2) * X_train_sin)

X_test_parabolic = np.linspace(-50, 50, 101)
X_test_sin = np.linspace(0, 2, 161)

## Sinus
Testowałem różne ustawienia, duży wpływ miało dostawienie kolejnej warstwy oraz ustawienie funkcji aktywacji na *tanh*. Sieć dość szybko minimalizowała błąd, dlatego zmniejszyłem liczbę iteracji i zwiększyłem *learning_rate*.

In [ ]:
# stworzenie modelu, odpowiednie warstwy
%matplotlib notebook
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

fig = plt.figure()
ax_1 = fig.add_subplot(2, 1, 1)
ax_1.set_title('Oryginalna funkcja')
ax_1.scatter(X_train_sin, Y_train_sin)
ax_2 = fig.add_subplot(2, 1, 2, sharex=ax_1)
fig.tight_layout(pad=2.0)
plt.ion()
fig.canvas.draw()

class PredictionCallback(tf.keras.callbacks.Callback):    
  def on_epoch_end(self, epoch, logs={}):
    if epoch % 100 == 0:
        Y_predicted_sin = self.model.predict(X_test_sin)
        ax_2.clear()
        ax_2.title.set_text('Aproksymacja')
        ax_2.scatter(X_test_sin, Y_predicted_sin)
        ax_2.set_xlabel(f"epoch = {epoch}, mse = {logs.get('loss')}") 
        fig.canvas.draw()
    
hidden_dim = 10
output_dim = 1

model = Sequential()
model.add(Dense(hidden_dim, activation=tf.nn.tanh))
model.add(Dense(hidden_dim, activation=tf.nn.tanh))
model.add(Dense(output_dim, activation=tf.nn.tanh))

model.compile(optimizer=optimizers.SGD(0.3),
              loss=tf.keras.losses.mse)

# trenowanie, funkcja fit
model.fit(X_train_sin, Y_train_sin, epochs=2000
          , callbacks=[PredictionCallback()])

## Parabola
Trudno było dobrać parametry w taki sposób, by funkcja w ogóle zbiegała. Podczas wielu testów, początkowo sieć minimalizowała błąd, ale po osiągnięciu mse $\approx$ 600000 nie było widać poprawy. Dopiero wyłączenie funkcji aktywacji (albo zmiana na ```tf.keras.activations.linear```) oraz zmiana *SGD* na *Adam* pozwoliła na poprawną aproksymację. Przy obecnych parametrach, gdybym zmienił z powrotem *Adam* na *SGD*, przybliżenie w ogóle by nie dawało poprawnych rezulatów, a mse = nan.

In [ ]:
fig = plt.figure()
ax_1 = fig.add_subplot(2, 1, 1)
ax_1.set_title('Oryginalna funkcja')
ax_1.scatter(X_train_parabolic, Y_train_parabolic)
ax_2 = fig.add_subplot(2, 1, 2, sharex=ax_1)
fig.tight_layout(pad=2.0)
plt.ion()
fig.canvas.draw()

class PredictionCallback(tf.keras.callbacks.Callback):    
  def on_epoch_end(self, epoch, logs={}):
    if epoch % 1000 == 0:
        Y_predicted_parabolic = self.model.predict(X_test_parabolic)
        ax_2.clear()
        ax_2.title.set_text('Aproksymacja')
        ax_2.scatter(X_test_parabolic, Y_predicted_parabolic)
        ax_2.set_xlabel(f"epoch = {epoch}, mse = {logs.get('loss')}") 
        fig.canvas.draw()
    
hidden_dim = 10
output_dim = 1

model = Sequential()
model.add(Dense(hidden_dim, activation=tf.nn.sigmoid))
model.add(Dense(hidden_dim))
model.add(Dense(output_dim))

model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.mse)

# trenowanie, funkcja fit
model.fit(X_train_parabolic, Y_train_parabolic, epochs=10000
          , callbacks=[PredictionCallback()])